# Chapter 8 文本数据
参考DataWhale：https://datawhalechina.github.io/joyful-pandas/build/html/%E7%9B%AE%E5%BD%95/ch8.html#str

In [1]:
import numpy as np
import pandas as pd

## 1. str对象
### 1.1 str对象的设计意图
str对象是Index或Series的属性，用于逐元素处理文本内容。对一个序列进行文本处理，首先需要获取其str对象。     
Python标准库中有str模块，pandas定义了许多用法一样的函数(31/50)。

In [2]:
#字母转为大写-python
var='abcd'
str.upper(var)

'ABCD'

In [3]:
#字母转为大写-Pandas
s=pd.Series(['abcd','efg','hi'])
s.str.upper()

0    ABCD
1     EFG
2      HI
dtype: object

### 1.2 [ ]索引器

In [4]:
#取出某个位置的元素
var[0]

'a'

In [5]:
#切片得到子串
var[-1:0:-2]

'db'

In [6]:
#pandas
s.str[0]

0    a
1    e
2    h
dtype: object

In [7]:
s.str[-1:0:-2]

0    db
1     g
2     i
dtype: object

### 1.3 string类型
字符串的数据存放类型，序列中至少有一个可迭代的对象（字符串、字典、列表）。对于一个可迭代对象，string类型的str对象和object类型的str对象返回结果可能不同。     
当序列类型为object时，对每一个元素进行[ ]索引。而string类型会把整个元素转为字面意思的字符串。

In [8]:
s=pd.Series([{1:'temp_1',2:'temp_2'}, ['a','b'], 0.5, 'my_string'])
s.str[1]

0    temp_1
1         b
2       NaN
3         y
dtype: object

In [9]:
s.astype('string').str[1]

0    1
1    '
2    .
3    y
dtype: string

object和string还有一个差别：string类型时Nullable类型，object不是。

In [10]:
#object: int
s=pd.Series(['a'])
s.str.len()

0    1
dtype: int64

In [11]:
#string: Int
s.astype('string').str.len()

0    1
dtype: Int64

In [12]:
#object: bool
s=='a'

0    True
dtype: bool

In [13]:
#object: boolean
s.astype('string')=='a'

0    True
dtype: boolean

全体元素是数值类型的序列，即使类型为object/category也不允许直接使用str属性。可以使用astype强制转换为string类型的Series。

In [14]:
s=pd.Series([12,345,6789])
s.astype('string').str[1]

0    2
1    4
2    7
dtype: string

## 2. 正则表达式基础
### 2.1 一般字符的匹配
正则表达式是一种按照某种正则模式，从左到右匹配字符串中内容的一种工具，对于一般的字符而言，它可以找到其所在的位置。

In [15]:
#re中的findall函数匹配出现过但不重叠的模式，第一个参数是正则表达式，第二个是待匹配的字符串
#在字符串中找出apple：
import re
re.findall(r'Apple', 'Apple! This is an Apple!')

['Apple', 'Apple']

### 2.2 元字符基础
.:   匹配除了换行符以外的任意字符；     
[ ]: 字符类，匹配方括号中包含的任意字符；         
[^]: 否定字符类，匹配方括号中不包含的任意字符；    
*:   匹配前面的子表达式零次或多次；      
+:   匹配前面的子表达式一次或多次；     
?:   匹配前面的子表达式零次或一次；     
{n, m}: 花括号，匹配前面字符至少n次，但是不超过m次；     
(xyz): 字符组，按照确切的顺序匹配字符xyz；     
|: 分支结构，匹配符号之前的字符或后面的字符；    
\: 转义符，还原元字符原来的含义；      
^: 匹配行的开始；     
$: 匹配行的结束；   

In [16]:
#.
re.findall(r'.', 'abc')

['a', 'b', 'c']

In [17]:
#[]
re.findall(r'[ac]', 'abc')

['a', 'c']

In [18]:
#[^]
re.findall(r'[^ac]', 'abc')

['b']

In [19]:
#{}匹配几次--字符长度
re.findall(r'[ab]{2}', 'aaaaaabbbbbb')

['aa', 'aa', 'aa', 'bb', 'bb', 'bb']

In [20]:
#|或
re.findall(r'aaa|bbb', 'aaaabbbb')

['aaa', 'bbb']

In [21]:
#\?*
#|左边匹配a+一个或零个\，右边匹配a*
re.findall(r'a\\?|a\*', 'aa?a*a')

['a', 'a', 'a', 'a']

In [22]:
#一个或零个a+所有
re.findall(r'a?.', 'abaacadaae')

['ab', 'aa', 'c', 'ad', 'aa', 'e']

### 2.3 简写字符集

\w: 匹配所有字母、数字、下划线：[a-z A-Z 0-9];  
\W: 匹配非字母和数字的字符：[^\w];       
\d: 匹配数字：[0-9];    
\D: 匹配非数字：[^\d];       
\s: 匹配空格符：[\t\n\f\r\p{Z}];           
\S: 匹配非空格符：[^\s];            
\B: 匹配一组非空字符开头或结尾的位置，不代表具体字符。

In [23]:
#两个字符且以s结尾
re.findall(r'.s', 'Apple! This Is an Apple')

['is', 'Is']

In [24]:
re.findall(r'\w{2}', '09 8? 7w c_ 9q p@')

['09', '7w', 'c_', '9q']

In [25]:
#规定了格式和长度
re.findall(r'\w\W\B', '09 8? 7w c_ 9q p@')

['8?', 'p@']

In [26]:
#空格及前后两个字符
re.findall(r'.\s.', 'Constant dropping wears the stone')

['t d', 'g w', 's t', 'e s']

In [27]:
re.findall(r'上海市(.{2,3}区)(.{2,3}路)(\d+号)', '上海市黄浦区方浜中路249号 上海市宝山区密山路5号')

[('黄浦区', '方浜中路', '249号'), ('宝山区', '密山路', '5号')]

## 3. 文本处理的五类操作
### 3.1 拆分
str.split函数参数：正则表达式、n（从左到右的最大拆分次数）、expand（是否展开为多个列）。

In [28]:
#定义拆分字符
s=pd.Series(['上海市黄浦区方浜中路249号', '上海市宝山区密山路5号'])
s.str.split('[市区路]')

0    [上海, 黄浦, 方浜中, 249号]
1       [上海, 宝山, 密山, 5号]
dtype: object

In [29]:
s.str.split('[市区路]', n=2, expand=True)

,0,1,2
0,上海,黄浦,方浜中路249号
1,上海,宝山,密山路5号


str.rsplit函数参数：正则表达式、n（从右到左的最大拆分次数）、expand（是否展开为多个列）。

### 3.2 合并
str.join：用某个连接符把Series中的字符串列表连接起来，如果列表中出现了非字符串元素则返回缺失值。

In [30]:
s=pd.Series([['a','b'], [1,'a'], [['a','b'],'c']])
s.str.join('-')

0    a-b
1    NaN
2    NaN
dtype: object

str.cat：用于合并两个序列，主要参数为连接符sep、连接形式join（默认以索引为键的左连接）、缺失值替代符号na_rep。

In [31]:
s1=pd.Series(['a','b'])
s2=pd.Series(['cat','dog'])
s1.str.cat(s2, sep='-')

0    a-cat
1    b-dog
dtype: object

In [32]:
s2.index=[1,2]
s1.str.cat(s2, sep='-', na_rep='?', join='outer')

0      a-?
1    b-cat
2    ?-dog
dtype: object

### 3.3 匹配
（1）str.contains：每个字符串是否包含正则模式的布尔序列。

In [33]:
s=pd.Series(['my cat','he is fat','railway station'])
s.str.contains('\s\wat')

0     True
1     True
2    False
dtype: bool

（2）str.startswith & str.endswith：返回每个字符串以给定模式为开始和结束的布尔序列，不支持正则表达式。

In [34]:
s.str.startswith('my')

0     True
1    False
2    False
dtype: bool

In [35]:
s.str.endswith('t')

0     True
1     True
2    False
dtype: bool

（3）str.match：用正则表达式检测开始或结束字符串的模式，返回每个字符串起始处是否符合给定正则模式的布尔序列。

In [36]:
s.str.match('m|h')

0     True
1     True
2    False
dtype: bool

In [37]:
s.str[::-1].str.match('ta[f|g]|n')

0    False
1     True
2     True
dtype: bool

In [38]:
s.str.contains('^[m|h]')

0     True
1     True
2    False
dtype: bool

In [39]:
s.str.contains('[f|g]at|n$')

0    False
1     True
2     True
dtype: bool

（4）str.find & str.rfind: 分别返回从左到右和从右到左第一次匹配的位置的索引，未找到则返回-1。不支持正则匹配，只能用于字符子串的匹配。

In [40]:
s=pd.Series(['This is an apple. That is not an apple'])
s.str.find('apple')

0    11
dtype: int64

In [41]:
s.str.rfind('apple')

0    33
dtype: int64

### 3.4 替换
（1）str.replace：使用字符串替换。

In [42]:
s=pd.Series(['a_1_b','c_?'])
s.str.replace('\d|\?', 'new', regex=True)

0    a_new_b
1      c_new
dtype: object

（2）对不同部分的不同替换，利用子组的方法分别处理，group(k)代表匹配到的第k个子组。

In [43]:
#先把地址拆分然后trans成英文
s = pd.Series(['上海市黄浦区方浜中路249号', '上海市宝山区密山路5号', '北京市昌平区北农路2号'])
pat='(\w+市)(\w+区)(\w+路)(\d+号)'
city={'上海市':'Shanghai', '北京市': 'Beijing'}
district={'昌平区':'CP District', '黄浦区':'HP District', '宝山区':'BS District'}
road={'方浜中路':'Mid Fangbin Road', '密山路':'Mishan Road', '北农路':'Beinong Road'}

def my_func(m):
    str_city=city[m.group(1)]
    str_district=district[m.group(2)]
    str_road=road[m.group(3)]
    str_no='No. '+m.group(4)[:-1]
    return ' '.join([str_city, str_district, str_road, str_no])

s.str.replace(pat, my_func, regex=True)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

（3）对子组命名。

In [44]:
pat='(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'

def my_func(m):
    str_city=city[m.group('市名')]
    str_district=district[m.group('区名')]
    str_road=road[m.group('路名')]
    str_no='No. '+m.group('编号')[:-1]
    return ' '.join([str_city, str_district, str_road, str_no])

s.str.replace(pat, my_func, regex=True)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

### 3.5 提取
（1）str.extract：返回具体元素的匹配操作，也是一种特殊的拆分操作。不会将分隔符去除！！

In [45]:
pat='(\w+市)(\w+区)(\w+路)(\d+号)'
s.str.extract(pat)

,0,1,2,3
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [46]:
#直接对新生成的DataFrame的列命名
pat='(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
s.str.extract(pat)

,市名,区名,路名,编号
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


（2）str.extractall：会把所有符合条件的模式全部匹配出来，如果存在多个结果，则以多级索引的方式存储。

In [47]:
s=pd.Series(['A135T15,A26S5','B674S2,B25T6'], index=['my_A','my_B'])
pat='[A|B](\d+)[T|S](\d+)'
s.str.extractall(pat)

0   1
     match         
my_A 0      135  15
     1       26   5
my_B 0      674   2
     1       25   6

In [48]:
pat_with_name='[A|B](?P<name1>\d+)[T|S](?P<name2>\d+)'
s.str.extractall(pat_with_name)

name1 name2
     match            
my_A 0       135    15
     1        26     5
my_B 0       674     2
     1        25     6

（3）str.findall与str.extractall功能类似，只是str.findall会把结果存入列表中，后者处理为多级索引。

In [49]:
s.str.findall(pat)

my_A    [(135, 15), (26, 5)]
my_B     [(674, 2), (25, 6)]
dtype: object

## 4. 常用字符串函数
### 4.1 字母型函数
（1）upper、lower、title、capitalize、swapcase：都用于字母的大小写转化。

In [50]:
s=pd.Series(['lower', 'CAPITALS', 'this is a sentence', 'SwApCaSe'])
s.str.upper()

0                 LOWER
1              CAPITALS
2    THIS IS A SENTENCE
3              SWAPCASE
dtype: object

In [51]:
s.str.lower()

0                 lower
1              capitals
2    this is a sentence
3              swapcase
dtype: object

In [52]:
s.str.title()

0                 Lower
1              Capitals
2    This Is A Sentence
3              Swapcase
dtype: object

In [53]:
s.str.capitalize()

0                 Lower
1              Capitals
2    This is a sentence
3              Swapcase
dtype: object

In [54]:
s.str.swapcase()

0                 LOWER
1              capitals
2    THIS IS A SENTENCE
3              sWaPcAsE
dtype: object

### 4.2 数值型函数
pd.to_numeric用于对字符格式的数值进行快速转换和筛选。         
pd.to_numeric函数参数：errors（非数值的处理模式）、downcast（非数值的转换类型）。       
对不能转换为数值的三种errors选项：raise（直接报错）、coerce（设为缺失）、ignore（保持原来的字符串）。

In [55]:
s=pd.Series(['1','2.2','2e','??','-2.1','0'])
pd.to_numeric(s, errors='ignore')

0       1
1     2.2
2      2e
3      ??
4    -2.1
5       0
dtype: object

In [56]:
pd.to_numeric(s, errors='coerce')

0    1.0
1    2.2
2    NaN
3    NaN
4   -2.1
5    0.0
dtype: float64

In [57]:
#利用coerce快速查看非数值型的行
s[pd.to_numeric(s, errors='coerce').isna()]

2    2e
3    ??
dtype: object

### 4.3 统计型函数
count：返回出现正则模式的次数；     
len：返回出现正则字符串的长度。

In [58]:
s=pd.Series(['cat rat fat at', 'get feed sheet heat'])
s.str.count('[r|f]at|ee')

0    2
1    2
dtype: int64

In [59]:
s.str.len()

0    14
1    19
dtype: int64

### 4.4 格式型函数
（1）除空型：strip（去除两侧空格）、rstrip（去除右侧空格）、lstrip（去除左侧空格）。

In [60]:
my_index=pd.Index([' col1','col2 ',' col3 '])
my_index.str.strip().str.len()

Int64Index([4, 4, 4], dtype='int64')

In [61]:
my_index.str.rstrip().str.len()

Int64Index([5, 4, 5], dtype='int64')

In [62]:
my_index.str.lstrip().str.len()

Int64Index([4, 5, 5], dtype='int64')

（2）填充型：pad选定字符串长度、填充的方向和填充内容。

In [63]:
s=pd.Series(['a','b','c'])
s.str.pad(4, 'left', '*')

0    ***a
1    ***b
2    ***c
dtype: object

In [64]:
s.str.pad(4, 'both', '*')

0    *a**
1    *b**
2    *c**
dtype: object

In [65]:
#左侧填充
s.str.rjust(5, '*')

0    ****a
1    ****b
2    ****c
dtype: object

In [66]:
s.str.ljust(5, '*')

0    a****
1    b****
2    c****
dtype: object

In [67]:
s.str.center(5, '*')

0    **a**
1    **b**
2    **c**
dtype: object

（3）补0的需求：zfill

In [68]:
s=pd.Series([7,155,303000]).astype('string')
s.str.zfill(6)

0    000007
1    000155
2    303000
dtype: string

## 5. 练习
### Ex1: 房屋信息数据集

In [69]:
df1=pd.read_excel('/Users/jie/Documents/Python/joyful-pandas-master/data/house_info.xls', 
                 usecols=['floor','year','area','price'])
df1.head(3)

,floor,year,area,price
0,高层（共6层）,1986年建,58.23㎡,155万
1,中层（共20层）,2020年建,88㎡,155万
2,低层（共28层）,2010年建,89.33㎡,365万


In [70]:
#1.
df1['year']=df1['year'].str.replace('年建', ' ', regex=True)
df1.head()

,floor,year,area,price
0,高层（共6层）,1986,58.23㎡,155万
1,中层（共20层）,2020,88㎡,155万
2,低层（共28层）,2010,89.33㎡,365万
3,低层（共20层）,2014,82㎡,308万
4,高层（共1层）,2015,98㎡,117万


In [71]:
#2.
pat1='(\D层)'
df1['Level']=df1['floor'].str.extract(pat1)
pat2='（共(\d+)层）'
df1['Highest']=df1['floor'].str.extract(pat2)
df1.drop(columns=['floor'])
df1.head()

,floor,year,area,price,Level,Highest
0,高层（共6层）,1986,58.23㎡,155万,高层,6
1,中层（共20层）,2020,88㎡,155万,中层,20
2,低层（共28层）,2010,89.33㎡,365万,低层,28
3,低层（共20层）,2014,82㎡,308万,低层,20
4,高层（共1层）,2015,98㎡,117万,高层,1


In [72]:
#3.
df1['avg_price']=pd.to_numeric(df1['price'].str.replace('万',' ',regex=True))/pd.to_numeric(df1['area'].str.replace('㎡',' ',regex=True))*10000
df1['avg_price']=df1['avg_price'].astype('int').astype('string')+'元/平米'
df1.head()

,floor,year,area,price,Level,Highest,avg_price
0,高层（共6层）,1986,58.23㎡,155万,高层,6,26618元/平米
1,中层（共20层）,2020,88㎡,155万,中层,20,17613元/平米
2,低层（共28层）,2010,89.33㎡,365万,低层,28,40859元/平米
3,低层（共20层）,2014,82㎡,308万,低层,20,37560元/平米
4,高层（共1层）,2015,98㎡,117万,高层,1,11938元/平米


### Ex2: 权利的游戏剧本数据集

In [73]:
df2=pd.read_csv('/Users/jie/Documents/Python/joyful-pandas-master/data/script.csv')
df2.head(3)

,Release Date,Season,Episode,Episode Title,Name,Sentence
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?


In [74]:
#1. 
df2.columns
#由于列名报错进行了检查，有空格的情况
df2.columns=df2.columns.str.strip()
df2.groupby(['Season','Episode'])['Sentence'].count()

Season    Episode   
Season 1  Episode 1     327
          Episode 10    266
          Episode 2     283
          Episode 3     353
          Episode 4     404
                       ... 
Season 8  Episode 2     405
          Episode 3     155
          Episode 4      51
          Episode 5     308
          Episode 6     240
Name: Sentence, Length: 73, dtype: int64

In [75]:
#2.
df2['Count']=df2['Sentence'].str.split('\s').str.len()
df2.groupby('Name')['Count'].mean().sort_values(ascending=False).head()

Name
male singer          109.000000
slave owner           77.000000
manderly              62.000000
lollys stokeworth     62.000000
dothraki matron       56.666667
Name: Count, dtype: float64

In [76]:
#3.
df2['Name']=df2['Name'].shift(-1)
df2['Count1']=df2['Sentence'].str.split('\?').str.len()-1
df2.groupby('Name')['Count1'].sum().sort_values(ascending=False).head()

Name
tyrion lannister    527
jon snow            374
jaime lannister     283
arya stark          265
cersei lannister    246
Name: Count1, dtype: int64